# David Kartchner

# Lab 10: Bokeh Solutions

# 11/7/16

In [1]:
import pandas as pd
import numpy as np
import bokeh as bk
import pickle
from matplotlib import pyplot as plt
%matplotlib inline
from pyproj import Proj, transform

/usr/local/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Problem 1

### Load FARS data from pickle files

In [2]:
# accidents10 = pd.read_pickle('fars_data/Accidents/accident10.pickle')
# accidents11 = pd.read_pickle('fars_data/Accidents/accident11.pickle')
# accidents12 = pd.read_pickle('fars_data/Accidents/accident12.pickle')
# accidents13 = pd.read_pickle('fars_data/Accidents/accident13.pickle')
# accidents14 = pd.read_pickle('fars_data/Accidents/accident14.pickle')
# a_list = [accidents10, accidents11, accidents12, accidents13, accidents14]


# drivers10 = pd.read_pickle('fars_data/Person/person10.pickle')
# drivers11 = pd.read_pickle('fars_data/Person/person11.pickle')
# drivers12 = pd.read_pickle('fars_data/Person/person12.pickle')
# drivers13 = pd.read_pickle('fars_data/Person/person13.pickle')
# drivers14 = pd.read_pickle('fars_data/Person/person14.pickle')
# d_list = [drivers10, drivers11, drivers12, drivers13, drivers14]
# drivers = pd.concat([drivers10, drivers11, drivers12, drivers13, drivers14])

# vehicles10 = pd.read_pickle('fars_data/Vehicle/vehicle10.pickle')
# vehicles11 = pd.read_pickle('fars_data/Vehicle/vehicle11.pickle')
# vehicles12 = pd.read_pickle('fars_data/Vehicle/vehicle12.pickle')
# vehicles13 = pd.read_pickle('fars_data/Vehicle/vehicle13.pickle')
# vehicles14 = pd.read_pickle('fars_data/Vehicle/vehicle14.pickle')
# v_list = [vehicles10, vehicles11, vehicles12, vehicles13, vehicles14]
# vehicles = pd.concat([vehicles10, vehicles11, vehicles12, vehicles13, vehicles14])

# with open('Pickle/id_to_state.pickle') as file:
#     id_to_state = pickle.load(file)
accidents = pd.read_pickle("final_accidents.pickle")
drivers = pd.read_pickle("final_drivers.pickle")

### Clean the data as follows:
1. Remove all test latitude/longitudes
2. Limit columns of accidents file to those we will use in our plot
3. Append data from vehicle file on whether people were speeding and call the created column "SPEEDING"
4. Map state index to letter code for each state

In [3]:
# clean_acc = []
# for a, v in zip(a_list, v_list):
#     a = a.loc[a.LATITUDE<77,['ST_CASE','STATE','LATITUDE','LONGITUD','HOUR','DAY','MONTH','YEAR','DRUNK_DR','WEATHER','FATALS']]
#     v.SPEEDREL.loc[v.SPEEDREL>=8]=0
#     v.SPEEDREL = (v.SPEEDREL>0).astype(np.int16)
#     v = v[['ST_CASE','SPEEDREL']].groupby('ST_CASE', as_index=False).max()
#     a = a.merge(v, on='ST_CASE')
#     a = a.rename(columns={'SPEEDREL':'SPEEDING'})
#     clean_acc.append(a)
# accidents = pd.concat(clean_acc)
# accidents.loc[accidents.STATE==0,"STATE"] = 49
# accidents.STATE = accidents.STATE.apply(lambda x: id_to_state[x]) 
# accidents.shape
# accidents.to_pickle("final_accidents.pickle")
# accidents.SPEEDING.sum()

/Users/davidspencerkartchner/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


44223

In [ ]:
a=accidents14
v=vehicles14

## Problem 2:
### Project Latitude and Longitude into X and Y coordinates usable in Bokeh

In [4]:
from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")
def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    Inputs:
    longitudes (array-like) : array of longitudes
    latitudes (array-like) : array of latitudes
    Example:
    x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals
# accidents["x"], accidents["y"] = convert(accidents.LONGITUD, accidents.LATITUDE)

In [9]:
accidents.to_pickle('final_accidents.pickle')

## Problem 3:
### Clean the Drivers data as follows:
1. Limit data to actual drivers (PER_TYP == 1)
2. Manually add year to data
3. Append speeding data to driver data

In [5]:
driver_list = []
for d, v, year in zip(d_list, v_list, range(2010,2015)):
    d["YEAR"] = year
    d = d.loc[d.PER_TYP==1,['ST_CASE','VEH_NO','AGE','DRINKING','YEAR']]
    v.SPEEDREL = (v.SPEEDREL>0).astype(np.int16)
    v = v[['ST_CASE','VEH_NO','SPEEDREL']]
    d = d.merge(v, on=['ST_CASE','VEH_NO'])
    driver_list.append(d)
drivers = pd.concat(driver_list)
drivers.to_pickle('final_drivers.pickle')

## Problem 4:
### Build base for Bokeh map

In [3]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import Figure
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool, ColumnDataSource

output_file("my_plot.html")
fig = figure(plot_width=1100, plot_height=650,
x_range=(-13000000, -7000000), y_range=(2750000, 6250000), webgl=True)
# fig.axis.visible = False
STAMEN_TONER_BACKGROUND = WMTSTileSource(
url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
attribution=(
'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
)
)
fig.add_tile(STAMEN_TONER_BACKGROUND)
show(fig)

## Problem 5:
### Add States to map as patches

In [5]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import Figure
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool, ColumnDataSource
with open('Pickle/us_states.pickle') as f:
    us_states = pickle.load(f)
    
fig = figure(plot_width=1100, plot_height=650)    
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]
map_xs, map_ys = convert(state_xs, state_ys)

# state_list = [state for state in np.unique(accidents.STATE)]
# state_totals = [accidents.loc[accidents.STATE==state].shape[0] for state in state_list]
# state_speeding = [float(accidents.loc[(accidents.STATE==state) & (accidents.SPEEDING>0)].shape[0])/tot for state, tot in zip(state_list, state_totals)]
# state_drinking = [float(accidents.loc[(accidents.STATE==state) & (accidents.DRUNK_DR>0)].shape[0])/tot for state, tot in zip(state_list, state_totals)]
# speeding_pc = [round(i*100, 2) for i in state_speeding]
# drinking_pc = [round(i*100, 2) for i in state_drinking]

# for i, j, k, l in zip(state_list, state_totals, speeding_pc, drinking_pc):
#     print i, j, k, l

border_data = {'x':map_xs, 'y':map_ys}#, 'state_name':state_list, 'state_speed':speeding_pc, 'state_drunk':drinking_pc, 'alpha':[0]*len(state_list)}
border_source = ColumnDataSource(data = border_data)

# fig = figure(plot_width=1100, plot_height=650, webgl=True)
bord_plot = fig.patches(xs='x',ys='y',source = border_source, line_width=1, fill_alpha=0) #,fill_alpha='alpha',line_alpha=0, line_width=1, hover_color='blue',hover_alpha=.5, hover_line_color='blue')
#fig.add_tools(HoverTool(renderers=[bord_plot], tooltips=[('State', '@state_name'), ("Speeding%","@state_speed"+"%"), ("Drunk%","@state_drunk"+"%")]))
show(fig)

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import Figure
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool, ColumnDataSource
with open('Pickle/us_states.pickle') as f:
    us_states = pickle.load(f)
    
fig = figure(plot_width=1100, plot_height=650)    
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]
map_xs, map_ys = convert(state_xs, state_ys)

border_data = {'x':map_xs, 'y':map_ys}
border_source = ColumnDataSource(data = border_data)

bord_plot = fig.patches(xs='x',ys='y',source = border_source, line_width=1, fill_alpha=0)

show(fig)

## Problem 6:
### Plot accidents based on cause and severity

In [8]:
speeding_accidents = accidents.loc[accidents.SPEEDING==1]
speeding_accidents["r"] = speeding_accidents.FATALS*100
drinking_accidents = accidents.loc[accidents.DRUNK_DR>1]
drinking_accidents["r"] = drinking_accidents.FATALS*100
other_accidents = accidents.loc[(accidents.SPEEDING==0)&(accidents.DRUNK_DR==0)]
other_accidents["r"] = other_accidents.FATALS*100


speeding = pd.DataFrame({'x_vals':speeding_accidents.x, 'y_vals':speeding_accidents.y, 'r':speeding_accidents.FATALS*100, 'color':'blue', 'alpha':0.5})
drinking = pd.DataFrame({'x_vals':drinking_accidents.x, 'y_vals':drinking_accidents.y, 'r':drinking_accidents.FATALS*100, 'color':'red', 'alpha':0.5})
other = pd.DataFrame({'x_vals':other_accidents.x, 'y_vals':other_accidents.y, 'r':other_accidents.FATALS*100, 'color':'yellow', 'alpha':0.5})

speeding_source = ColumnDataSource(data=speeding)
drinking_source = ColumnDataSource(data=drinking)
other_source = ColumnDataSource(data=other)

speed_cir = fig.circle(x='x_vals', y='y_vals',radius='r', fill_color='color',fill_alpha='alpha',source=speeding_source,line_color='color')
drinking_cir = fig.circle(x='x_vals', y='y_vals',radius='r', fill_color='color',fill_alpha='alpha',source=drinking_source,line_color='color')
other_cir = fig.circle(x='x_vals', y='y_vals',radius='r', fill_color='color',fill_alpha='alpha',source=other_source,line_color='color')


/Users/davidspencerkartchner/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/davidspencerkartchner/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/davidspencerkartchner/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [9]:
show(fig)